In [3]:
#1.install and import dependencies
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [4]:
#2.Using MP Holistic to get the keypoints
mp_holistic = mp.solutions.holistic #Holistic model
mp_drawing = mp.solutions.drawing_utils #Drawing utilities

In [5]:
#functions declarations
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #Read feed
        ret, frame  = cap.read()
        
        #Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(image, results)
        #print(results)
        
        #draw landmarks
        draw_styled_landmarks(image, results)
        
        #Show to screen
        cv2.imshow("OpenCV Feed", image)
        
        #Break gracefully when q key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
draw_landmarks(frame, results)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))


In [7]:
#3. Extract the keypoint values
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)
#print(pose)

NameError: name 'results' is not defined

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    # 468*3 + 33*4 + 21*3 + 21*3 
    return np.concatenate([pose, face, lh, rh])

In [ ]:
result_test  = extract_keypoints(results)
#np.save('0',result_test)
#np.load('0.npy')

In [8]:
# 4.Setup folders to store the keypoints
#Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_DATA')#the parent folder's path will starts with MP_DATA

#Actions that we try to detect
actions = np.array(['hello','thanks','iloveyou','ryoki tenkai'])

#Thirty videos worth of data
no_sequences = 30

#Each of the video will have 30 frames in length
sequence_length = 30

#Folder start
start_folder = 30

In [9]:
for action in actions:
    #dirmax  = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(no_sequences):
        try: 
            #make directory MP_DATA/hello/0
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            #if there's exception just ignore it and keep going
            pass

In [10]:
# 5.Collecting keypoints values for training and testing 
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #new loop 
    #loop through actions
    for action in actions:
        #loop through sequences aka videos
        for sequence in range(no_sequences):
            #loop through video length aka sequence length
            for frame_num in range(sequence_length):
                #Read feed
                ret, frame  = cap.read()
                
                #Make detections
                image, results = mediapipe_detection(frame, holistic)
                
                #draw landmarks
                draw_styled_landmarks(image, results)
                
                #new apply wait logic to give breaks
                if frame_num == 0:
                    cv2.putText(image, "Starting Collection", (120, 200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, "Collecting frames for {} Video Number {}".format(action, sequence), (15, 12),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0, 255), 1, cv2.LINE_AA)
                    #Show to screen
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, "Collecting frames for {} Video Number {}".format(action, sequence), (15, 12),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0, 255), 1, cv2.LINE_AA) 
                    
                cv2.imshow("Sign Language Recognition", image)
                #new export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                #Break gracefully when q key is pressed
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    

D:\Workshop 2 Project\action_recognition_model\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [12]:
cap.release()
cv2.destroyAllWindows()

In [13]:
#6. Preprocess data and create labels and features
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
label_map = {label: num for num, label in enumerate(actions)}
label_map

{np.str_('hello'): 0,
 np.str_('thanks'): 1,
 np.str_('iloveyou'): 2,
 np.str_('ryoki tenkai'): 3}

In [15]:
# 30 sequences = 30 videos recorded , labels = y labels
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH,action))).astype(int):
        #to store each of the video
        window = []
        #to append each of the frame together to form a complete video
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [16]:
np.array(labels).shape

(120,)

In [17]:
x = np.array(sequences)
x.shape

(120, 30, 1662)

In [18]:
y = to_categorical(labels).astype(int)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.05)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(114, 30, 1662) (6, 30, 1662) (114, 4) (6, 4)


In [20]:
#7. Build and train LSTM Neural Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Input
from tensorflow.keras.callbacks import TensorBoard

In [21]:
#to store the training logs, can view the progress using cmd: tensorboard --logdir=.
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [22]:
model = Sequential()
#have to return the sequences so that the next layer can use them
# Specify the input using Input layer
model.add(Input(shape=(30, 1662)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
#softmax means the max value would be 1 and 1 would be allocated to each of the different actions as probability
model.add(Dense(actions.shape[0],activation='softmax'))
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [24]:
#train the model **this will takes time
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - categorical_accuracy: 1.0000 - loss: 2.0422e-04
Epoch 2/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - categorical_accuracy: 1.0000 - loss: 2.4106e-04
Epoch 3/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - categorical_accuracy: 1.0000 - loss: 2.1135e-04
Epoch 4/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - categorical_accuracy: 1.0000 - loss: 2.3398e-04
Epoch 5/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - categorical_accuracy: 1.0000 - loss: 2.1604e-04
Epoch 6/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - categorical_accuracy: 1.0000 - loss: 2.3660e-04
Epoch 7/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - categorical_accuracy: 1.0000 - loss: 1.8813e-04
Epoch 8/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - categorical_accuracy: 1.0000 - loss: 1.8223e-04
Epoch 9/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - categorical_accuracy: 1.0000 - loss: 1.8867e-04
Epoch 10/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - categorical_accuracy: 1.0000 - loss: 

KeyboardInterrupt: 

In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 64)         │       442,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 30, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,790,126 (6.83 MB)

 Trainable params: 596,708 (2.28 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,193,418 (4.55 MB)

In [27]:
#8.Make predictions
res = model.predict(x_test)
actions[np.argmax(res[4])]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 534ms/step


np.str_('hello')

In [30]:
actions[np.argmax(y_test[3])]

np.str_('ryoki tenkai')

In [31]:
#9.Save the model 
model.save("recognition_model_v2.keras")

In [71]:
#delete the model
del model

In [81]:
model.load_weights("recognition_model_v1.keras")

In [32]:
#10. Evaluation using confusion matrix and accuracy
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [33]:
yhat = model.predict(x_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [34]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 2]],

       [[4, 0],
        [0, 2]],

       [[5, 1],
        [0, 0]],

       [[4, 0],
        [1, 1]]])

In [36]:
accuracy_score(ytrue, yhat)

0.8333333333333334

In [35]:
#11.Test in real time
from scipy import stats

In [38]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
#function to draw the probability bars on each of the actions
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90 +num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return output_frame

In [39]:
#plot the output figure
plt.figure(figsize=(9,9))
plt.imshow(prob_viz(res, actions, image, colors))

TypeError: only length-1 arrays can be converted to Python scalars

<Figure size 900x900 with 0 Axes>

In [ ]:
#testing the model
#new variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #Read feed
        ret, frame  = cap.read()
        
        #Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(image, results)
        #print(results)
        
        #draw landmarks
        draw_styled_landmarks(image, results)
        
        #prediction logic 
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        #get the latest 30 keypoints
        sequence = sequence[-30:]
        
        #make prediction every 30 frame
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print("Predicted Action: " + actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            #Viz logic: to check whether the last prediction is the same as the current one
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
            if len(sentence) > 1:
                sentence = sentence[-1:]
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        #Show to screen
        cv2.imshow("Sign Language Recognition", image)
        
        #Break gracefully when q key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()